### Create a Langchain application
Create a new Python program and copy this code into it.

In [1]:
from utilities.llms import llm_3

c:\Users\Ai Sukmoren\Desktop\langchain_demo\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.azure_openai.AzureOpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_community.tools import YouTubeSearchTool
from langchain_community.graphs import Neo4jGraph
from langchain.chains import RetrievalQA
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain.schema import StrOutputParser
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.schema.messages import HumanMessage, SystemMessage
import os

In [2]:
question = "What is Neo4j?"

response = (llm.invoke(question)).content

print(response)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
template = PromptTemplate(template="""
        You are a cockney fruit and vegetable seller.
        Your role is to assist your customer with their fruit and vegetable needs.
        Respond using cockney rhyming slang.

        Tell me about the following fruit: {fruit}
        """, input_variables=["fruit"])

response = (llm.invoke(template.format(fruit="apple"))).content

print(response)

In [ ]:
template = PromptTemplate.from_template("""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Tell me about the following fruit: {fruit}
Tell me about the following vegatable: {vegatable}
""")

llm_chain = LLMChain(
    llm=llm,
    prompt=template,
    verbose=True
)

input_variables = {"fruit": "apple","vegatable":"cucumber"}

response = llm_chain.invoke(input_variables)

print(response['text'])

In [ ]:
llm = ChatOpenAI(temperature= 0, model_name="gpt-4",api_key=openai_api_key)

template = PromptTemplate.from_template("""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Tell me about the following fruit: {fruit}
""")

llm_chain = LLMChain(
    llm=llm,
    prompt=template,
    output_parser=StrOutputParser()
)

response = llm_chain.invoke({"fruit": "apple"})

print(response['text'])

In [ ]:
llm = ChatOpenAI(temperature= 0, model_name="gpt-4",api_key=openai_api_key)

template = PromptTemplate.from_template("""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Output JSON as {{"description": "your response here"}}

Tell me about the following fruit: {fruit}
""")

llm_chain = LLMChain(
    llm=llm,
    prompt=template,
    output_parser=SimpleJsonOutputParser()
)

response = llm_chain.invoke({"fruit": "apple"})

print(response['text'])

## chatmodels

In [ ]:
prompt = PromptTemplate(
    template="""You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Question: {question}
""",
    input_variables=["question"],
)

chat_chain = LLMChain(llm=llm, prompt=prompt)

response = chat_chain.invoke({"question": "What is the weather like?"})

print(response)

In [ ]:
llm = ChatOpenAI(temperature= 0, model_name="gpt-4",api_key=openai_api_key)


current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Polzeath", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

prompt = PromptTemplate(
    template="""You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Context: {context}
Question: {question}
""",
    input_variables=["context", "question"],
)

chain = LLMChain(llm=llm, prompt=prompt)

response = chain.invoke(
    {
        "context": current_weather,
        "question": "What is the weather like?"
     
    }
)
print(response['text'])

### Chat_history

In [14]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain.embeddings import AzureOpenAIEmbeddings


# Load environment variables from .env file
load_dotenv()

AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")


llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    openai_api_version="2023-03-15-preview",
    azure_deployment="gpt4_1106",
    openai_api_key=AZURE_OPENAI_API_KEY,
)

llm_3 = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    openai_api_version="2023-03-15-preview",
    azure_deployment="gpt-35-turbo-0613",
    openai_api_key=AZURE_OPENAI_API_KEY,
)

embeddings = AzureOpenAIEmbeddings(
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            openai_api_version="2023-03-15-preview",
            deployment="embedada",
            openai_api_key=AZURE_OPENAI_API_KEY,
            )

In [7]:
prompt = PromptTemplate(
    template="""
You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Chat History: {chat_history}
Context: {context}
Question: {question}
""",
    input_variables=["chat_history", "context", "question"],
)

memory = ConversationBufferMemory(
    memory_key="chat_history", input_key="question", return_messages=True
)

chat_chain = LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=True)

current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""
    
while True:
    question = input("> ")
    if question == 'exit':
        break
    else:
        response = chat_chain.invoke({"context": current_weather, "question": question})
        print(response["text"])



> Entering new LLMChain chain...
Prompt after formatting:

You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Chat History: []
Context: 
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }
Question: hi


> Finished chain.
Hey there, bro! Stoked to chat about the gnarly waves! What's the surf sitch you're curious about? 🤙


> Entering new LLMChain chain...
Prompt after formatting:

You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Chat History: [HumanMessage(content='hi'), AIMessage(content="Hey there, bro! Stoked to chat about the gnarly waves! What's the surf sitch you're curious about? 🤙")]
Context: 
    {
        "surf": [
            {"beach": 

### Agent

In [16]:
prompt = PromptTemplate(
    template="""
    You are a movie expert. You find movies from a genre or plot.

    ChatHistory:{chat_history}
    Question:{input}
    """,
    input_variables=["chat_history", "input"],
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

youtube = YouTubeSearchTool()

tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=chain.run,
        return_direct=True
    ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word 'trailer'. Return a link to a YouTube video.",
        func=youtube.run,
        return_direct=True
    )
]
agent_prompt = hub.pull("hwchase17/react-chat")

agent = create_react_agent(llm, tools, agent_prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    max_interations=3,
    verbose=True,
    handle_parse_errors=True
    )

while True:
    question = input("> ")
    if question == 'exit':
        break
    else:
        response = agent_executor.invoke({"input": question})
        print(response['output'])



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
Final Answer: Hello! How can I assist you today?
```

> Finished chain.
{'input': 'hi', 'chat_history': [HumanMessage(content='hi'), AIMessage(content='Hello! How can I assist you today?\n```')], 'output': 'Hello! How can I assist you today?\n```'}


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
Final Answer: Determining the "best" horror movie of 2018 can be subjective as it depends on individual tastes and criteria for what makes a horror movie great. However, one film that received critical acclaim and was popular among audiences is "Hereditary" directed by Ari Aster. It's known for its intense atmosphere, strong performances, particularly by Toni Collette, and its chilling and unsettling narrative. Other notable horror films from 2018 include "A Quiet Place" by John Krasinski and "Halloween" which is a sequel to the original 1978 film. Each of these films has its own un

### Connecting with Neo4j

In [15]:
graph = Neo4jGraph(
    url="neo4j+s://e6e3d283.databases.neo4j.io",
    username="neo4j",
    password="H9u9aKqOVYyHOVRDyFLcMCf7RcBiMyit_wvgol6IrRw"
)

Unable to retrieve routing information


ValueError: Could not connect to Neo4j database. Please ensure that the url is correct

In [18]:
graph = Neo4jGraph(
    url="neo4j://4.194.131.167/:7687",
    username="neo4j",
    password="P@ssword"
)

print(graph.schema)

Node properties are the following:
Document {language: STRING, fileName: STRING, carBrand: STRING, carModel: STRING},Manual {language: STRING, fileName: STRING, carBrand: STRING, carModel: STRING, title: STRING, page: INTEGER},Section {title: STRING, fileName: STRING, page: INTEGER},ManualParent {title: STRING, carBrand: STRING, carModel: STRING, language: STRING, page: STRING, id: INTEGER, text: STRING, fileName: STRING},ManualChild {title: STRING, page: STRING, embedding: LIST, carBrand: STRING, carModel: STRING, language: STRING, id: STRING, text: STRING},ManualImage {id: STRING, filePath: STRING},CarPart {name: STRING},TermAndCondition {fileName: STRING, language: STRING, id: INTEGER, text: STRING, embedding: LIST},TandCChild {id: INTEGER, text: STRING, embedding: LIST, fileName: STRING}
Relationship properties are the following:

The relationships are the following:
(:Document)-[:HAS_SECTION]->(:Manual),(:Document)-[:HAS_SECTION]->(:Section),(:Document)-[:HAS_CONTENT]->(:TermAndCo

In [25]:
graph = Neo4jGraph(
    url="bolt://18.207.205.88:7687",
    username="neo4j",
    password="inclinations-odors-laser"
)

In [26]:
print(graph.schema)

Node properties are the following:
Document {language: STRING, fileName: STRING, carBrand: STRING, carModel: STRING},Manual {language: STRING, fileName: STRING, carBrand: STRING, carModel: STRING, title: STRING, page: INTEGER},Section {title: STRING, fileName: STRING, page: INTEGER},ManualParent {title: STRING, carBrand: STRING, carModel: STRING, language: STRING, page: STRING, id: INTEGER, text: STRING, fileName: STRING},ManualChild {title: STRING, page: STRING, embedding: LIST, carBrand: STRING, carModel: STRING, language: STRING, id: STRING, text: STRING},ManualImage {id: STRING, filePath: STRING},CarPart {name: STRING},TermAndCondition {fileName: STRING, language: STRING, id: INTEGER, text: STRING, embedding: LIST},TandCChild {id: INTEGER, text: STRING, embedding: LIST, fileName: STRING}
Relationship properties are the following:

The relationships are the following:
(:Document)-[:HAS_SECTION]->(:Manual),(:Document)-[:HAS_SECTION]->(:Section),(:Document)-[:HAS_CONTENT]->(:TermAndCo

### Retrievers

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.schema import Document


In [27]:
embedding_provider = OpenAIEmbeddings(api_key=openai_api_key)

movie_plot_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    url="bolt://18.207.205.88:7687",
    username="neo4j",
    password="inclinations-odors-laser",
    index_name="moviePlots",
    embedding_node_property="embedding",
    text_node_property="plot",
)

result = movie_plot_vector.similarity_search("A movie where aliens land and attack earth.",k=1)
for doc in result:
    print(doc.metadata["title"], "-", doc.page_content,"TMDB",doc.metadata["url"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import Neo4jVector


llm = ChatOpenAI(temperature= 0, model_name="gpt-4",api_key=openai_api_key)

embedding_provider = OpenAIEmbeddings(api_key=openai_api_key)

movie_plot_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    url="bolt://18.207.205.88:7687",
    username="neo4j",
    password="inclinations-odors-laser",
    index_name="moviePlots",
    embedding_node_property="embedding",
    text_node_property="plot",
)

plot_retriever = RetrievalQA.from_llm(
    llm=llm,
    retriever=movie_plot_vector.as_retriever(),
    verbose=True,
    return_source_documents=True
)

query = "A movie where aliens land and attack earth"

result = plot_retriever.invoke(
    {"query": query}
)

print(result)

In [ ]:
pip install langchainhub

In [ ]:
llm = ChatOpenAI(temperature= 0, model_name="gpt-4",api_key=openai_api_key)

prompt = PromptTemplate(
    template="""
    You are a movie expert. You find movies from a genre or plot.

    ChatHistory:{chat_history}
    Question:{input}
    """,
    input_variables=["chat_history", "input"],
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chat_chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

youtube = YouTubeSearchTool()

embedding_provider = OpenAIEmbeddings(api_key=openai_api_key)

movie_plot_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    url="bolt://18.207.205.88:7687",
    username="neo4j",
    password="inclinations-odors-laser", 
    index_name="moviePlots",
    embedding_node_property="embedding",
    text_node_property="plot",
)

plot_retriever = RetrievalQA.from_llm(
    llm=llm,
    retriever=movie_plot_vector.as_retriever()
)


def run_retriever(query):
    results = plot_retriever({"query":query})
    return str(results)


tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=chat_chain.run,
        return_direct=True,
    ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word 'trailer'. Return a link to a YouTube video.",
        func=youtube.run,
        return_direct=True,
    ),
    Tool.from_function(
        name="Movie Plot Search",
        description="For when you need to compare a plot to a movie. The question will be a string. Return a string.",
        func=run_retriever,
        return_direct=True,
    ),
]

agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    max_interations=3,
    verbose=True,
    handle_parse_errors=True,
)

while True:
    q = input("> ")
    if q.lower() in ["exit", "quit", "end"]:
        print("Exiting the chat...")
        break
    response = agent_executor.invoke({"input": q})
    print(response["output"])

### The Cypher QA Chain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature= 0, model_name="gpt-3.5-turbo",api_key=openai_api_key)

graph = Neo4jGraph(
    url="bolt://18.207.205.88:7687",
    username="neo4j",
    password="inclinations-odors-laser", 
)

CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True
)

question = "How many movies has Tom Hanks directed?"

cypher_chain.invoke({question})

### Providing Specific Instructions

In [19]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
llm = ChatOpenAI(temperature= 0, model_name="gpt-4",api_key=openai_api_key)

def dmac_qa(question):
    """
    Use the  Vector Search Index
    to augment the response from the LLM
    """
    embeddings = OpenAIEmbeddings()

    damac_vectorstore = FAISS.load_local("damac_database", embeddings)

    template = """
    You are an expert female lawyer who is specialized in ansewering various ploblem regard to the agreement between 2 parties.

    If no data is returned, do not attempt to answer the question.
    Do not include any explanations or apologies in your responses. 


    Question: {question}
    context: {context}
    """

    summary_template = PromptTemplate(input_variables=['question','context'],template= template)
    
    retriever = damac_vectorstore.as_retriever()
    dmac_qa = RetrievalQA.from_llm(
        llm=llm, 
        retriever=retriever,
        prompt=summary_template,
        verbose=True
        )
    result = dmac_qa.run(question)
    return str(result)

dmac_qa(question = "Who is involved in this agreement and what is the reason for this agreement")



> Entering new RetrievalQA chain...

> Finished chain.


'The parties involved in this agreement are SUPSUMPHAN CO., LTD. and DARIA DATA CENTRE AND CLOUD SERVICES CO., LTD. The reason for this agreement is the lease of land owned by SUPSUMPHAN CO., LTD. to DARIA DATA CENTRE AND CLOUD SERVICES CO., LTD. for the purpose of establishing its first data center in Thailand.'

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature= 0, model_name="gpt-4",api_key=openai_api_key)

graph = Neo4jGraph(
    url="bolt://18.207.205.88:7687",
    username="neo4j",
    password="inclinations-odors-laser", 
)

CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
For movie titles that begin with "The", move "the" to the end, For example "The 39 Steps" becomes "39 Steps, The" or "The Matrix" becomes "Matrix, The".

If no data is returned, do not attempt to answer the question.
Only respond to questions that require you to construct a Cypher statement.
Do not include any explanations or apologies in your responses.

Examples:

Find movies and genres:
MATCH (m:Movie)-[:IN_GENRE]->(g)
RETURN m.title, g.name

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True
)

question = "What genre of film is Toy Story?"
cypher_chain.invoke({question})